In [1]:
# !/bin/python3
# https://adventofcode.com/2022/day/18

%load_ext lab_black

In [2]:
from utils import read_input

In [3]:
def make_map(scan):
    cubes = {}
    for x, y, z in scan:
        if x not in cubes:
            cubes[x] = {}
        if y not in cubes[x]:
            cubes[x][y] = []
        cubes[x][y].append(z)
    return cubes

In [4]:
def part_a(scan):
    cubes = make_map(scan)
    count = 0

    sides = [(-1, 0, 0), (0, -1, 0), (0, 0, -1), (1, 0, 0), (0, 1, 0), (0, 0, 1)]
    for x_offset, y_offset, z_offset in sides:
        for x, y_values in cubes.items():
            for y, z_values in y_values.items():
                for z in z_values:
                    if z + z_offset not in cubes.get(x + x_offset, {}).get(
                        y + y_offset, []
                    ):
                        count += 1

    return count

In [5]:
def get_points(scan):
    x_points = {}
    y_points = {}
    z_points = {}

    for x, y, z in scan:
        x_points = update_points(x_points, x, y, z)
        y_points = update_points(y_points, y, x, z)
        z_points = update_points(z_points, z, x, y)

    return x_points, y_points, z_points


def update_points(points, a, b, c):
    if (b, c) not in points:
        points[(b, c)] = []

    points[(b, c)].append(a)
    return points


def get_potential_holes(scan):
    potential = set()
    points = {}

    sides = [(-1, 0, 0), (0, -1, 0), (0, 0, -1), (1, 0, 0), (0, 1, 0), (0, 0, 1)]
    for x, y, z in scan:
        for x_offset, y_offset, z_offset in sides:
            holes = find_holes(scan, x, y, z)
            if not holes:
                continue

            if ((x, y, z)) not in points:
                points[(x, y, z)] = set()

            points[(x, y, z)].update(holes)
            potential.update(holes)
    return points, potential


def find_holes(scan, x, y, z):
    holes = set()
    sides = [(-1, 0, 0), (0, -1, 0), (0, 0, -1), (1, 0, 0), (0, 1, 0), (0, 0, 1)]
    for x_offset, y_offset, z_offset in sides:
        curr_x = x + x_offset
        curr_y = y + y_offset
        curr_z = z + z_offset
        if [curr_x, curr_y, curr_z] not in scan:
            holes.add((curr_x, curr_y, curr_z))
    return holes


def find_edges(scan, potential, x_points, y_points, z_points):
    holes = set()
    edges = set()

    for point in potential:
        if point in holes or point in edges:
            continue

        group = set()
        check = [point]
        edge = False
        while check:
            x, y, z = check.pop()
            if (x, y, z) in group:
                continue
            group.add((x, y, z))

            if (
                is_point_on_edge(x_points, x, y, z)
                or is_point_on_edge(y_points, y, x, z)
                or is_point_on_edge(z_points, z, x, y)
            ):
                edges.update(group)
                edge = True
                break

            new_points = find_holes(scan, *point)
            check.extend(new_points)
        if not edge:
            holes.update(group)

    return edges


def is_point_on_edge(points, a, b, c):
    if (b, c) not in points:
        return False

    if a < min(points[(b, c)]):
        return True

    if a > max(points[(b, c)]):
        return True

    return False

In [6]:
def part_b(scan):
    x_points, y_points, z_points = get_points(scan)
    points, potential_holes = get_potential_holes(scan)
    edges = find_edges(scan, potential_holes, x_points, y_points, z_points)

    count = 0
    for potential in points.values():
        for point in potential:
            if point in edges:
                count += 1
    return count

In [7]:
scan = read_input(
    parent=__vsc_ipynb_file__, line_delimiter=",", val_type=int, sample="a"
)


print("part a:", part_a(scan))
print("part b:", part_b(scan))

part a: 10
part b: 10


In [8]:
scan = read_input(
    parent=__vsc_ipynb_file__, line_delimiter=",", val_type=int, sample="b"
)

print("part a:", part_a(scan))
print("part b:", part_b(scan))

part a: 64
part b: 58


In [9]:
scan = read_input(parent=__vsc_ipynb_file__, line_delimiter=",", val_type=int)

print("part a:", part_a(scan))
print("part b:", part_b(scan))

part a: 4444
part b: 2530
